In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [12]:
df = pd.read_csv(r'C:\Users\MedTech\Desktop\02.12.25\test_patients_bsk_20251202.csv', encoding='windows-1251', sep=';')

In [5]:
df

,emias_id,last_date_of_attachment,start_date_benefit,end_date_benefit,benefit_code,diag_code
0,29 337 809,22.10.2014,05.11.2019,13.11.2026,760,I48.0
1,19 291 047,30.01.2013,03.02.2021,08.11.2026,760,I48.0
2,16 708 278,09.03.2017,27.11.2019,03.11.2026,760,I48.0
3,260 757 866,01.01.2014,29.11.2019,13.11.2026,760,I48.1
4,10 330 331,16.09.2016,26.11.2019,02.11.2026,760,I48.1
...,...,...,...,...,...,...
322,28 758 239,22.06.2016,13.11.2023,12.11.2026,760,I48.0
323,30 000 014 037 648,21.02.2023,18.08.2023,18.11.2026,760,I48.1
324,30 011 784,10.10.2018,30.11.2023,13.11.2026,760,I48.1
325,2 004 615 991,15.01.2018,09.11.2023,15.11.2026,760,I48.1


In [ ]:
import pandas as pd

# Загружаем данные
df = pd.read_csv(
    r'C:\Users\MedTech\Desktop\02.12.25\test_patients_bsk_20251202.csv',
    encoding='windows-1251',
    sep=';'
)

# Список столбцов, где нужно преобразовать даты
date_cols = [
    'last_date_of_attachment',
    'start_date_benefit',
    'end_date_benefit'
]

# Конвертация строковых дат "DD.MM.YYYY" → datetime → формат ClickHouse
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%d.%m.%Y', errors='coerce').dt.strftime('%Y-%m-%d')

# Проверим результат
print(df.head())


In [23]:
cool = df

In [ ]:
cool

In [25]:
# Сохраняем файл в CSV
output_path = r'C:\Users\MedTech\Desktop\02.12.25\test_patients_bsk_20251202_converted.csv'

cool.to_csv(
    output_path,
    index=False,          # не добавлять индекс
    sep=';',              # как в исходном файле
    encoding='utf-8-sig'  # корректная UTF-8 с BOM, идеально для ClickHouse и Excel
)

print("Файл успешно сохранён:", output_path)

Файл успешно сохранён: C:\Users\MedTech\Desktop\02.12.25\test_patients_bsk_20251202_converted.csv
